<h1>Laboratorio N° 07: Monitorización de Base de Datos mediante Auditoría</h1>

<h2>Ejercicio N° 01: Aplicando auditorias</h2>

Paso 1: Crear una auditoría del servidor con las siguientes propiedades
- Name: activity_audit
- Queue delay: 1000 ms
- On failure: continue
- Target: file
- Target file path: D:\Auditoria

In [22]:
USE master;
GO
CREATE SERVER AUDIT activity_audit 
    TO FILE (FILEPATH='D:\Auditoria')
    WITH (QUEUE_DELAY = 1000, ON_FAILURE = CONTINUE);
GO


Commands completed successfully.

: Msg 15530, Level 16, State 1, Line 2
auditoría con el nombre "activity_audit" ya existe.

Total execution time: 00:00:00.401

Paso 2: Activar la auditoria del servidor creada.

In [23]:
ALTER SERVER AUDIT activity_audit WITH (STATE = ON);
GO

Commands completed successfully.

Total execution time: 00:00:00.008

Paso 3: Crear una especificación de auditoría del servidor con las siguientes propiedades.
- Name: audit_logins
- Audit: activity_audit
- Action groups: SUCCESSFUL_LOGIN_GROUP

In [24]:
CREATE SERVER AUDIT SPECIFICATION audit_logins
FOR SERVER AUDIT activity_audit
ADD (SUCCESSFUL_LOGIN_GROUP)
WITH (STATE = ON);
GO

: Msg 15530, Level 16, State 1, Line 1
especificación de auditoría con el nombre "audit_logins" ya existe.

Total execution time: 00:00:00.001

Paso 4: Activar la especificación de auditoria del servidor creada.

In [25]:
ALTER SERVER AUDIT SPECIFICATION audit_logins WITH (STATE = ON);
GO

Commands completed successfully.

Total execution time: 00:00:00.005

Paso 5: Crear una especificación de auditoría de base de datos en la base de datos salesapp1 con las siguientes propiedades:
- Name: employees_change_audit
- Audit: activity_audit
- Actions:
    - INSERT on HR.Employees by public
    - UPDATE on HR.Employees by public

In [26]:
USE salesapp1;
GO
CREATE DATABASE AUDIT SPECIFICATION employees_change_audit
FOR SERVER AUDIT activity_audit
ADD (INSERT,UPDATE ON DATABASE::salesapp1 BY public),
ADD (SELECT ON SCHEMA::HR BY public)
WITH (STATE = ON);
GO

Commands completed successfully.

: Msg 15530, Level 16, State 1, Line 2
especificación de auditoría con el nombre "employees_change_audit" ya existe.

Total execution time: 00:00:00.005

Paso 6: Activar la especificación de auditoría de base de datos creada.

: Msg 33073, Level 16, State 1, Line 1
No se encuentra el elemento especificación de auditoría 'employees_change_audit' o no tiene los permisos necesarios.

Total execution time: 00:00:00.001

Paso 7: Ejecutar el siguiente código

In [27]:
USE salesapp1;

UPDATE HR.Employees SET mgrid = 3 WHERE empid = 9;

INSERT HR.Employees
(lastname, firstname, title, titleofcourtesy, birthdate, hiredate, address, city, region, postalcode, country, phone, mgrid)
SELECT lastname, firstname, title, titleofcourtesy, birthdate, hiredate, address, city, region, postalcode, country, phone, mgrid
FROM HR.Employees WHERE empid = 9;

EXECUTE AS USER = 'marketing_user'
UPDATE HR.Employees SET mgrid = 5 WHERE empid = 9;
REVERT
GO

(1 row affected)

(1 row affected)

: Msg 15517, Level 16, State 1, Line 10
No se puede ejecutar como la entidad de seguridad de base de datos porque la entidad "marketing_user" no existe, este tipo de entidad de seguridad no se puede suplantar o el usuario no tiene permiso.

Total execution time: 00:00:00.012

Paso 8: Escribir una consulta utilizando la función de sistema sys.fn_get_audit_file para devolver todos los datos de auditoría desde los archivos en D:\Auditoria. Filtrar los datos para que solo la actividad relacionada a la sesión actual sea visualizada.

Paso 8: Desahbilitar la auditoría de servidor activity_audit.